In [22]:
# packages
import os, sys
sys.path.append(os.path.dirname(os.getcwd()) + '/source')
import numpy as np
import pickle
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import requests
family_dir = os.path.dirname(os.getcwd())

In [2]:
# model parameter
δ = .01
η = .032
median = 1.75/1000 # unit: celcius per gigatonne
h_hat = .15
σ_n = 1.2
γ_low = .012
γ_high = .024
γ_base = .018
ξ = σ_n/h_hat*δ*η
e_simul = pickle.load(open(family_dir+'/data/e_simul','rb'))

# Emission trajectory simualtion
Use the above values, simulate emission based on $\tau_2 = 0$, $\xi = 8\times\delta\times\eta = .00256$ and $ \xi = 4\times\delta\times\eta = .00128$.

The simulation starts with initial reserve of 1500 and time span is 500 years.

In [3]:
# plot emission for 200 years
e_base = e_simul['base']

In [57]:
from plots import fig1
fig = fig1(
    e_base, 
    legend_list= [r'$\tau_2 = 0$', r'$\xi = 0.00256$', r'$\xi = 0.00128$'],
    xylabel = ('Years', 'Emission'),
    title = "Figure 3"
)
fig.show()

# $h^*$ plot

The $h^*$ is computed in the following way:
$$
    h^* = - \cfrac{e (v_y)'\Theta\sigma_n}{\xi_m}
$$
Then rescaled by $-(v_y)'\Theta\sigma_n$ and divided by $\cfrac{\gamma}{1000}$ 

In [5]:
from utilities import compute_h_hat
e_quad = {x: e_base[x] for x in ['xi1', 'xi05']}
h_hat = {x: compute_h_hat(e_quad[x], γ_base, xi) for x, xi in [['xi05', .5*ξ], ['xi1', ξ]]}

In [60]:
fig = fig1(
    h_hat,
    legend_list= [r'$\xi = 0.00128$', r'$\xi = 0.00256$'],
    xylabel = ('Years', 'Drift Distortion'),
    title = "Figure 4"
)
fig.show()

# Implied Standard Deviation

The implied standard deviation is computed as follows:
$$
    \cfrac{\sqrt{\sum{e_t^2}}\cdot \sigma_n \cdot [-(v_y)'\Theta]}{(\sum{e_t})\cdot\frac{\gamma(1-\eta)}{1000}} =  \cfrac{\sqrt{\sum{e_t^2}}}{\sum{e_t}}\cdot \sigma_n\cdot \text{median}\cdot 1000
$$

In [54]:
from utilities import compute_std
implied_std = {}
for γ, damage in [[γ_low, 'low'], [γ_high, 'high'], [γ_base, 'base']]:
    emission = e_simul[damage]
    for ξ_m, mdl_misspfy in [[.5*ξ, 'xi05'], [ξ, 'xi1']]:
        emission_selected = emission[mdl_misspfy]
        std_key = r'γ={}'.format(γ) + ', ' + r'ξ_m = {}'.format(ξ_m)
        implied_std[std_key] = [format(compute_std(emission_selected, time), '.2f') for time in [25, 50, 100, 200]]

In [55]:
from prettytable import PrettyTable
from IPython.display import display, Math, Latex
print('Table 2: Implied Standard Deviation')
tab = PrettyTable()
tab.field_names = ['', '25 years', '50 years', '100 years', '200 years']
for key, stds in implied_std.items():
    tab.add_row([key]+stds)
print(tab)

Table 2: Implied Standard Deviation
+------------------------+----------+----------+-----------+-----------+
|                        | 25 years | 50 years | 100 years | 200 years |
+------------------------+----------+----------+-----------+-----------+
| γ=0.012, ξ_m = 0.00128 |   0.42   |   0.30   |    0.21   |    0.15   |
| γ=0.012, ξ_m = 0.00256 |   0.42   |   0.30   |    0.21   |    0.16   |
| γ=0.024, ξ_m = 0.00128 |   0.42   |   0.30   |    0.21   |    0.15   |
| γ=0.024, ξ_m = 0.00256 |   0.42   |   0.30   |    0.21   |    0.15   |
| γ=0.018, ξ_m = 0.00128 |   0.42   |   0.30   |    0.21   |    0.15   |
| γ=0.018, ξ_m = 0.00256 |   0.42   |   0.30   |    0.21   |    0.15   |
+------------------------+----------+----------+-----------+-----------+
